In [182]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [183]:
import sqlite3
import numpy as np
import pandas as pd
from src.tables import table_to_df
from src.utils import reduce_mem_usage

conn = sqlite3.connect('../data/vast_old.db')

In [184]:
rel_df = table_to_df('reliability_ts', conn)
rel_df = reduce_mem_usage(rel_df)

rel_df.set_index('machine_id', inplace=True)
rel_df.sort_index(inplace=True)

rel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1497207 entries, 162 to 12852
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype
---  ------       --------------    -----
 0   reliability  1497207 non-null  int16
 1   timestamp    1497207 non-null  int32
dtypes: int16(1), int32(1)
memory usage: 20.0 MB


In [185]:
ts_idx = rel_df.timestamp.unique()
machine_ids = rel_df.index.unique()

df = pd.DataFrame(columns=[machine_ids], index=ts_idx, dtype='UInt16')
df.sort_index(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26378 entries, 1691935057 to 1694165858
Columns: 1957 entries, (162,) to (12852,)
dtypes: UInt16(1957)
memory usage: 147.9 MB


In [186]:

# machine_id = 12113
# machine_id = 8506
# ts = rel_df.loc[machine_id, 'timestamp']
# vals = rel_df.loc[machine_id, 'reliability']
# if ts.shape:
#     ts = ts.values
#     vals = vals.values
# df.loc[ts, machine_id] = vals


In [187]:
%%time
for machine_id in machine_ids:
    try:
        ts = rel_df.loc[machine_id, 'timestamp']
        vals = rel_df.loc[machine_id, 'reliability']
        if ts.shape:
            ts = ts.values
            vals = vals.values
        df.loc[ts, machine_id] = vals
    except Exception as e:
        print(machine_id, e)

CPU times: total: 1.62 s
Wall time: 1.62 s


In [189]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26378 entries, 1691935057 to 1694165858
Columns: 1957 entries, (162,) to (12852,)
dtypes: UInt16(1957)
memory usage: 148.9 MB


In [204]:
# by timestamp
def explode_table(id_col, col, table, conn):    
    ts_idx = pd.read_sql('SELECT timestamp FROM timestamp_tbl', conn).timestamp.values
    raw_df = pd.read_sql(f'SELECT timestamp, {id_col}, {col} FROM {table}', conn).set_index(id_col)
    raw_df.sort_index(inplace=True)
    raw_df = reduce_mem_usage(raw_df)
    raw_df[col] = raw_df[col].convert_dtypes()
    ids = raw_df.index.unique()
    
    df = pd.DataFrame(columns=[ids], index=ts_idx, dtype=raw_df[col].dtype)    
    
    for idx in ids:
        ts = raw_df.loc[idx, 'timestamp']
        vals = raw_df.loc[idx, col]
        if ts.shape:
            ts = ts.values
            vals = vals.values
        df.loc[ts, idx] = vals
        
    df.fillna(method='ffill', inplace=True)
    return df

In [205]:
df = explode_table('machine_id', 'online', 'online_ts', conn)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26975 entries, 1691935057 to 1694165858
Columns: 1957 entries, (162,) to (12852,)
dtypes: Int8(1957)
memory usage: 101.9 MB


In [201]:
df.fillna(0, inplace=True)
df.astype(np.uint8).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26975 entries, 1691935057 to 1694165858
Columns: 1957 entries, (162,) to (12852,)
dtypes: uint8(1957)
memory usage: 51.6 MB


In [ ]:
ts_rent = pd.read_sql('SELECT DISTINCT timestamp FROM rent_ts', conn).timestamp
ts_rent.shape

In [ ]:
ts_rel = pd.read_sql('SELECT DISTINCT timestamp FROM reliability_ts', conn).timestamp
ts_rel.shape

In [ ]:
ts_rent[~ts_rent.isin(ts_rel)]

In [ ]:
df = explode_table('id', 'rented', 'rent_ts', conn)
df.info()

In [ ]:
df[274].dropna().eq(0).sum()

In [ ]:
for col in df:
    if df[col].eq(0).any():
        print(col)

In [ ]:
arr = df[12760].fillna(0).values

In [ ]:
%timeit non_zero_idx = np.nonzero(arr); _ = arr[non_zero_idx]

In [ ]:
%timeit _ = arr[non_zero_idx]

In [ ]:
from pandas.core.dtypes.common import is_integer_dtype, is_float_dtype

